In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import layers

In [2]:
x_train=np.load('train_image.npy').reshape(60000,28,28,1)
label_train=np.load('train_label.npy')
y_train=np.zeros((60000,10))
for i in range(60000):
    y_train[i,int(label_train[i])]=1

In [3]:
class optical_layer(keras.layers.Layer):
    def __init__(self, units=200):
        super(optical_layer, self).__init__()
        self.units = units

    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-2], input_shape[-1]),
            initializer="random_normal",
            trainable=True,
        )


    def call(self, inputs):
        tmp=tf.complex(inputs,0.0)
        tmp_w=tf.complex(self.w,0.0)
        output=tf.math.abs(tf.signal.ifft2d(tf.signal.fft2d(tmp)* tmp_w))
        return output

In [5]:
num_classes = 10
input_shape = (28, 28,1)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        optical_layer(),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
optical_layer_1 (optical_lay (None, 28, 28, 1)         28        
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
Total params: 7,878
Trainable params: 7,878
Non-trainable params: 0
_________________________________________________________________


In [7]:
#test tool
batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/5
422/422 [==============================] - 24s 56ms/step - loss: 0.3453 - accuracy: 0.8974 - val_loss: 0.2365 - val_accuracy: 0.9328
Epoch 2/5
422/422 [==============================] - 24s 56ms/step - loss: 0.2926 - accuracy: 0.9134 - val_loss: 0.2243 - val_accuracy: 0.9342
Epoch 3/5
422/422 [==============================] - 23s 55ms/step - loss: 0.2652 - accuracy: 0.9218 - val_loss: 0.2057 - val_accuracy: 0.9390
Epoch 4/5
422/422 [==============================] - 24s 56ms/step - loss: 0.2482 - accuracy: 0.9258 - val_loss: 0.1983 - val_accuracy: 0.9427
Epoch 5/5
422/422 [==============================] - 26s 62ms/step - loss: 0.2334 - accuracy: 0.9306 - val_loss: 0.1856 - val_accuracy: 0.9452
